<h1>Non-standard aminoacids</h1>

In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import re
import requests
from bs4 import BeautifulSoup

import time

from rdkit import Chem
from rdkit.Chem import Descriptors

from collections import Counter

<h2>DRAMP</h2>

In [189]:
dramp = pd.read_csv('dramp.csv', index_col='Unnamed: 0').drop(columns=['pdb', 'comments'])
dramp.head()

,id,seq,len,target,topology,nterm,cterm,mod,stereo,db
51,DRAMP00795,GIPCGESCVFIPCITAAIGCSCKSKVCYRN,30,[Ref.21596752]Gram-positive bacteria: Staphylo...,Cyclic,Free,Free,There are three disulfide bonds between Cys4 a...,L,DRAMP
52,DRAMP00798,GIPCGESCVFIPCITGAIGCSCKSKVCYRN,30,[Ref.21596752]Gram-positive bacteria: Staphylo...,Cyclic,Free,Free,There are three disulfide bonds between Cys4 a...,L,DRAMP
67,DRAMP01064,AWKLFDDGV,9,[Ref.21882228]Gram-positive bacteria: Staphylo...,Linear,Free,Free,NaN,L,DRAMP
171,DRAMP03024,LKLKSIVSWAKKVL,14,[Ref.10667861] Gram-positive bacteria: Staphyl...,Linear,Free,Amidation,Free,L,DRAMP
327,DRAMP01607,GLFDIIKKIAESF,13,Gram-positive bacteria: Bacillus cereus (MIC=1...,Linear,Free,Amidation,Free,L,DRAMP


In [190]:
dramp['mod'] = dramp['mod'].fillna('Free')
dramp_nonclassic = dramp[dramp['mod'] != 'Free']
dramp_nonclassic.shape

(1012, 10)

In [191]:
dramp_nonclassic['target'] = dramp_nonclassic['target'].fillna('Not available')
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.strip())
dramp_nonclassic = dramp_nonclassic[dramp_nonclassic['target'] != 'Not available']
dramp_nonclassic = dramp_nonclassic[dramp_nonclassic['target'] != 'No MICs found in DRAMP database']
dramp_nonclassic.shape

/tmp/ipykernel_291872/1940622863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dramp_nonclassic['target'] = dramp_nonclassic['target'].fillna('Not available')
/tmp/ipykernel_291872/1940622863.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.strip())


(641, 10)

In [192]:
target_list = list(dramp_nonclassic['target'])
print(*target_list, sep='\n')

[Ref.21596752]Gram-positive bacteria: Staphylococcus aureus ATCC12600, Enterococcus faecalis ATCC 47077 (less active);##Gram-negative bacteria: Escherichia coli ATCC 700926 (MIC=1.1 µM), Pseudomonas aeruginosa ATCC 39018 (MIC=2.7 µM), Klebsiella pneumonia ATCC 13883 (MIC=4.7 µM).
[Ref.21596752]Gram-positive bacteria: Staphylococcus aureus ATCC12600, Enterococcus faecalis ATCC 47077 (less active);##Gram-negative bacteria: Escherichia coli ATCC 700926 (MIC=1.3 µM), Pseudomonas aeruginosa ATCC 39018 (MIC=1.9 µM), Klebsiella pneumonia ATCC 13883 (MIC=1.5 µM).
[Ref.29859288] Gram-negative bacteria:Escherichia coli (ATCC 25922)(MIC=25 ± 0 μg/ml);ESBL-producing Escherichia coli(MIC=33.3 ± 14.4μg/ml);NDM-1 producing Acinetobacter baumannii(MIC=20.8 ± 7.2 μg/ml);##Gram-positive bacteria:Staphylococcus aureus (ATCC 25923)(MIC=133.3 ± 57.7μg/ml);##Fungi:Candida albicans (ATCC 90028)(MIC>400μg/ml)
[Ref.29040295] Gram-positive bacteria: Staphylococcus aureus (MIC=8 μg/ml; MBC=16 μg/ml), Methicillin

In [193]:
def parse_target(target_line):

    if target_line == 'Not available':
        return [None]
    
    if type(target_line) != str:
        return [None]

    # Tumor cells: mode
    if target_line[:12] == 'Tumor cells:':
        target_line = target_line[12:].strip()
        return target_line.split(';')
    

    # Cancer cell lines: mode
    if target_line.find('Cancer cell lines:') !=-1:
        border = target_line.find('Cancer cell lines:')+18
        target_line = target_line[border:].strip()
        return target_line.split(',')
    
    # Cancer:
    if target_line.find('Cancer:') !=-1:
        border = target_line.find('Cancer:')+7
        target_line = target_line[border:].strip()
        target_line = target_line.split('##')[0]
        return target_line.split(';')

    return [None]

dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(parse_target)
dramp_nonclassic = dramp_nonclassic.explode('target').dropna(subset=['target'])

In [194]:
target_list = list(dramp_nonclassic['target'])
print(*target_list, sep='\n')

C8166 (IC50=15.3 µg/ml)
 Molt-4 (IC50=24.3 µg/ml)
 BIU-87 (IC50=20.5 µg/ml)
 T24 (IC50=35.4 µg/ml).
HeLa (IC50=0.15±0.08 nM)
 PC-12 (IC50=7.71±0.24 nM)
 MCF-7 (IC50=11.20±0.02 nM)
 K562 (IC50=15.80±0.09 nM).##Non-cancer cell lines: Vero (IC50=2.79±0.05 nM) and Madin-Darby canine kidney cells (IC50=3.92±0.01 nM).
HeLa (IC50=0.15±0.08 nM)
 PC-12 (IC50=7.71±0.24 nM)
 MCF-7 (IC50=11.20±0.02 nM)
 K562 (IC50=15.80±0.09 nM).##Non-cancer cell lines: Vero (IC50=2.79±0.05 nM) and Madin-Darby canine kidney cells (IC50=3.92±0.01 nM).
BGC-823 (IC50=1.75±0.05 µM)
 MM96L (IC50=4.91±0.04 µM)
 HeLa (IC50=15.5±0.06 µM).##Non-cancer cell line: HFF-1 (IC50=3.19±0.01 µM).
BGC-823 (IC50=2.91±0.06 µM)
 MM96L (IC50=1.03±0.03 µM)
 HeLa (IC50=6.35±0.31 µM).##Non-Cancer cell line: HFF-1 (IC50=1.76±0.12 µM).
BGC-823 (IC50=2.91±0.06 µM)
 MM96L (IC50=1.03±0.03 µM)
 HeLa (IC50=6.35±0.31 µM)##Non-Cancer cell line: HFF-1 (IC50=1.76±0.12 µM).
MM96L (IC50=2.51±0.03 µM)
 HeLa (IC50=5.24±0.40 µM).##Non-Cancer cell line: H

In [195]:
dramp_nonclassic.shape

(1763, 10)

In [196]:
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.split('##'))
dramp_nonclassic = dramp_nonclassic.explode(column='target')
dramp_nonclassic.shape

(1773, 10)

In [197]:
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.split('and'))
dramp_nonclassic = dramp_nonclassic.explode(column='target')
dramp_nonclassic.shape

(1775, 10)

In [198]:
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.strip())
target_list = list(dramp_nonclassic['target'])
print(*target_list, sep='\n')

C8166 (IC50=15.3 µg/ml)
Molt-4 (IC50=24.3 µg/ml)
BIU-87 (IC50=20.5 µg/ml)
T24 (IC50=35.4 µg/ml).
HeLa (IC50=0.15±0.08 nM)
PC-12 (IC50=7.71±0.24 nM)
MCF-7 (IC50=11.20±0.02 nM)
K562 (IC50=15.80±0.09 nM).
Non-cancer cell lines: Vero (IC50=2.79±0.05 nM)
Madin-Darby canine kidney cells (IC50=3.92±0.01 nM).
HeLa (IC50=0.15±0.08 nM)
PC-12 (IC50=7.71±0.24 nM)
MCF-7 (IC50=11.20±0.02 nM)
K562 (IC50=15.80±0.09 nM).
Non-cancer cell lines: Vero (IC50=2.79±0.05 nM)
Madin-Darby canine kidney cells (IC50=3.92±0.01 nM).
BGC-823 (IC50=1.75±0.05 µM)
MM96L (IC50=4.91±0.04 µM)
HeLa (IC50=15.5±0.06 µM).
Non-cancer cell line: HFF-1 (IC50=3.19±0.01 µM).
BGC-823 (IC50=2.91±0.06 µM)
MM96L (IC50=1.03±0.03 µM)
HeLa (IC50=6.35±0.31 µM).
Non-Cancer cell line: HFF-1 (IC50=1.76±0.12 µM).
BGC-823 (IC50=2.91±0.06 µM)
MM96L (IC50=1.03±0.03 µM)
HeLa (IC50=6.35±0.31 µM)
Non-Cancer cell line: HFF-1 (IC50=1.76±0.12 µM).
MM96L (IC50=2.51±0.03 µM)
HeLa (IC50=5.24±0.40 µM).
Non-Cancer cell line: HFF-1 (IC50=1.55±0.09 µM).
MM96

In [199]:
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.replace('Leukemia cell lines', ''))
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.replace('(NCI)', '[NCI]'))
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.replace('(Bc1-xL dependent)', '[Bc1-xL dependent]'))
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.replace('(Bcl-2 dependent)', '[Bcl-2 dependent]'))
dramp_nonclassic['target'] = dramp_nonclassic['target'].apply(lambda x: x.strip())

In [200]:
def parse_ic50(text):

    match = re.search(r'\((.*)\)', text)
    if match:
        return match.group(1)
    
    return None

dramp_nonclassic['ic50'] = dramp_nonclassic['target'].apply(parse_ic50)
print(*list(dramp_nonclassic['ic50'].unique()), sep='\n')

IC50=15.3 µg/ml
IC50=24.3 µg/ml
IC50=20.5 µg/ml
IC50=35.4 µg/ml
IC50=0.15±0.08 nM
IC50=7.71±0.24 nM
IC50=11.20±0.02 nM
IC50=15.80±0.09 nM
IC50=2.79±0.05 nM
IC50=3.92±0.01 nM
IC50=1.75±0.05 µM
IC50=4.91±0.04 µM
IC50=15.5±0.06 µM
IC50=3.19±0.01 µM
IC50=2.91±0.06 µM
IC50=1.03±0.03 µM
IC50=6.35±0.31 µM
IC50=1.76±0.12 µM
IC50=2.51±0.03 µM
IC50=5.24±0.40 µM
IC50=1.55±0.09 µM
IC50=1.32±0.15 µM
IC50=3.10±0.06 µM
IC50=10.21±0.43 µM
IC50=2.38±0.09 µM
IC50=34.4 μg/ml
IC50=24.2 µg/ml
IC50=11.4 µg/ml
IC50=25.2 µg/ml
IC50=28 µg/ml
IC50=18 µg/ml
IC50=20.97 µM
IC50=18.7 µM
IC50=66.4 µM
IC50=44.7 µM
IC50=83.49 µM
IC50=894.7 µM
IC50=15.54 µM
IC50=85.41 µM
IC50=18.9 µM
IC50=27.72 µM
IC50=224.8 µM
IC50=25.04 µM
IC50=9.01 µM
IC50=34.74 µM
IC50=10.52 µM
IC50=11 µM
IC50=41.55 µM
IC50=9.39 µM
IC50=34.56 μM
IC50=25.13 μM
IC50=38.56 μM
IC50=33.23 μM
IC50=58.62 μM
IC50=1.4 μM
IC50=4.1 μM
IC50=7.02 µM
IC50=13.87 µM
IC50=8.56 µM
IC50=13.66 µM
LC50=100 µM
LC50>50 µM
LC50=2.5 µM
LC50=4.5 µM
40% inhibition at 1 µM
44

In [201]:
def clean_ic50(s):

    if s is None:
        return None

    whitelist = ('IC50', 'LC50', 'LD50', 'CC50', 'EC50','ED50' ,'50%')
    
    for w in whitelist:
        if s.find(w) !=-1:
            return s
        
    return None

dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(clean_ic50)
dramp_nonclassic = dramp_nonclassic.dropna(subset=['ic50'])
dramp_nonclassic.shape

(1412, 11)

In [202]:
dramp_nonclassic['cell_line'] = dramp_nonclassic['target'].apply(lambda x: x.split('(')[0].strip())
print(*list(dramp_nonclassic['cell_line']), sep='\n')

C8166
Molt-4
BIU-87
T24
HeLa
PC-12
MCF-7
K562
Non-cancer cell lines: Vero
Madin-Darby canine kidney cells
HeLa
PC-12
MCF-7
K562
Non-cancer cell lines: Vero
Madin-Darby canine kidney cells
BGC-823
MM96L
HeLa
Non-cancer cell line: HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
MM96L
HeLa
Non-Cancer cell line: HFF-1
MM96L
HeLa
Non-Cancer cell line: HFF-1
BGC-823
MM96L
HeLa
Non-cancer cell line: HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
C8166
C8166
Molt-4
C8166
Molt-4
BIU-87
T24
MCF-7
Bcap-37
MDA-MB-231
MCF-7
Bcap-37
MDA-MB-231
MCF-7
MDA-MB-231
Bcap-37
MCF-7
MDA-MB-231
Bcap-37
MCF-7
MDA-MB-231
Bcap-37
MCF-7
MDA-MB-231
Bcap-37
NCI-H157
U251MG
PC-3
MDA-MB-435s
HMEC-1
Human melanoma A2058
Human melanoma A2058
Human squamous lung carcinoma NCI-H157
Human breast adenocarcinoma MDA-MB-435S
Human prostate adenocarcinoma PC-3
Human glioblastoma U251-MG
Human breast adenocarcinoma MCF-7
B1

In [203]:
dramp_nonclassic['cell_line'] = dramp_nonclassic['cell_line'].apply(lambda x: x.replace('Non-cancer cell lines:', '').strip())
dramp_nonclassic['cell_line'] = dramp_nonclassic['cell_line'].apply(lambda x: x.replace('Non-cancer cell line:', '').strip())
print(*list(dramp_nonclassic['cell_line']), sep='\n')

C8166
Molt-4
BIU-87
T24
HeLa
PC-12
MCF-7
K562
Vero
Madin-Darby canine kidney cells
HeLa
PC-12
MCF-7
K562
Vero
Madin-Darby canine kidney cells
BGC-823
MM96L
HeLa
HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
MM96L
HeLa
Non-Cancer cell line: HFF-1
MM96L
HeLa
Non-Cancer cell line: HFF-1
BGC-823
MM96L
HeLa
HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
BGC-823
MM96L
HeLa
Non-Cancer cell line: HFF-1
C8166
C8166
Molt-4
C8166
Molt-4
BIU-87
T24
MCF-7
Bcap-37
MDA-MB-231
MCF-7
Bcap-37
MDA-MB-231
MCF-7
MDA-MB-231
Bcap-37
MCF-7
MDA-MB-231
Bcap-37
MCF-7
MDA-MB-231
Bcap-37
MCF-7
MDA-MB-231
Bcap-37
NCI-H157
U251MG
PC-3
MDA-MB-435s
HMEC-1
Human melanoma A2058
Human melanoma A2058
Human squamous lung carcinoma NCI-H157
Human breast adenocarcinoma MDA-MB-435S
Human prostate adenocarcinoma PC-3
Human glioblastoma U251-MG
Human breast adenocarcinoma MCF-7
B16-F10
D122
B16-F10
D122
CEM-WT
NGP
IMR-5
CL-1
MDA-MB-231
22Rv1
HeLa
A549
PC-3
HepG2
EJ
L51

In [204]:
dramp_nonclassic = dramp_nonclassic.dropna(subset=['ic50'])
dramp_nonclassic.shape

(1412, 12)

In [205]:
def get_units(s):

    # μmol/mL
    for u in ('μmol/mL', 'µmol/mL', 'μmol/ml', 'µmol/ml'):
        if s.find(u) !=-1:
            return 'microM/ml'

    # microM [μ,µ]
    for u in ('μM', 'µM', 'μmol/L', 'µmol/L','μm', 'µm'):
        if s.find(u) !=-1:
            return 'microM'
        
    # microg/ml [μ,µ]
    for u in ('μg/ml', 'µg/ml', 'μg/mL', 'µg/mL', 'µg/m', 'mg/L'):
        if s.find(u) !=-1:
            return 'microg/ml'
        
    # nM
    for u in ('nM',):
        if s.find(u) !=-1:
            return 'nanoM'
        
    # mM
    for u in ('mM',):
        if s.find(u) !=-1:
            return 'mM'
        
    # M
    for u in ('M',):
        if s.find(u) !=-1:
            return 'M'
        
    
    # mg/mL
    for u in ('mg/ml', 'mg/mL'):
        if s.find(u) !=-1:
            return 'mg/ml'
        
    return s

dramp_nonclassic['units'] = dramp_nonclassic['ic50'].apply(get_units)
dramp_nonclassic['units'].value_counts()


units
microM       1332
microg/ml      65
nanoM          12
mM              3
Name: count, dtype: int64

In [206]:
get_units('IC50=0.8 mM')

'mM'

In [207]:
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace('>', '=').replace('<', '=').replace('==', '='))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.split('±')[0])
print(*list(dramp_nonclassic['ic50'].unique()), sep='\n')


IC50=15.3 µg/ml
IC50=24.3 µg/ml
IC50=20.5 µg/ml
IC50=35.4 µg/ml
IC50=0.15
IC50=7.71
IC50=11.20
IC50=15.80
IC50=2.79
IC50=3.92
IC50=1.75
IC50=4.91
IC50=15.5
IC50=3.19
IC50=2.91
IC50=1.03
IC50=6.35
IC50=1.76
IC50=2.51
IC50=5.24
IC50=1.55
IC50=1.32
IC50=3.10
IC50=10.21
IC50=2.38
IC50=34.4 μg/ml
IC50=24.2 µg/ml
IC50=11.4 µg/ml
IC50=25.2 µg/ml
IC50=28 µg/ml
IC50=18 µg/ml
IC50=20.97 µM
IC50=18.7 µM
IC50=66.4 µM
IC50=44.7 µM
IC50=83.49 µM
IC50=894.7 µM
IC50=15.54 µM
IC50=85.41 µM
IC50=18.9 µM
IC50=27.72 µM
IC50=224.8 µM
IC50=25.04 µM
IC50=9.01 µM
IC50=34.74 µM
IC50=10.52 µM
IC50=11 µM
IC50=41.55 µM
IC50=9.39 µM
IC50=34.56 μM
IC50=25.13 μM
IC50=38.56 μM
IC50=33.23 μM
IC50=58.62 μM
IC50=1.4 μM
IC50=4.1 μM
IC50=7.02 µM
IC50=13.87 µM
IC50=8.56 µM
IC50=13.66 µM
LC50=100 µM
LC50=50 µM
LC50=2.5 µM
LC50=4.5 µM
50% inhibition at 1 µM
=50% apoptosis at 20 µM
50% apoptosis at 50 µM
LC50=3 μmol/L
LC50=6 μmol/L
LC50=7.4 µM
LC50=7.5 µM
IC50=20.3
IC50=21.2
IC50=21.6
ED50=0.39µM
ED50=3.4µM
ED50=3.8µM
ED50=9µ

In [208]:
def clean_ic50_2(s):

    
    # μmol/mL
    for u in ('μmol/mL', 'µmol/mL', 'μmol/ml', 'µmol/ml'):
        if s.find(u) !=-1:
            return s.replace(u, '')

    # microM [μ,µ]
    for u in ('μM', 'µM', 'μmol/L', 'µmol/L','μm', 'µm'):
        if s.find(u) !=-1:
            return s.replace(u, '')
        
    # microg/ml [μ,µ]
    for u in ('μg/ml', 'µg/ml', 'μg/mL', 'µg/mL', 'µg/m', 'mg/L'):
        if s.find(u) !=-1:
            return s.replace(u, '')
        
    # nM
    for u in ('nM',):
        if s.find(u) !=-1:
            return s.replace(u, '')
        
     # nM
    for u in ('mM',):
        if s.find(u) !=-1:
            return s.replace(u, '')
        
    # nM
    for u in ('M',):
        if s.find(u) !=-1:
            return s.replace(u, '')
        
    
    # mg/mL
    for u in ('mg/ml', 'mg/mL'):
        if s.find(u) !=-1:
            return s.replace(u, '')
        
    return s
    

dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(clean_ic50_2)
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace(' at ', '='))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace('=50%', '50%'))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace('(4.8-5.5)', ''))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace('(0.82-1.1)', ''))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace('(32-40)', ''))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace('(2.0-2.5)', ''))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.replace('(DMSO', ''))
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.split('=')[-1].strip())
print(*list(dramp_nonclassic['ic50'].unique()), sep='\n')

15.3
24.3
20.5
35.4
0.15
7.71
11.20
15.80
2.79
3.92
1.75
4.91
15.5
3.19
2.91
1.03
6.35
1.76
2.51
5.24
1.55
1.32
3.10
10.21
2.38
34.4
24.2
11.4
25.2
28
18
20.97
18.7
66.4
44.7
83.49
894.7
15.54
85.41
18.9
27.72
224.8
25.04
9.01
34.74
10.52
11
41.55
9.39
34.56
25.13
38.56
33.23
58.62
1.4
4.1
7.02
13.87
8.56
13.66
100
50
2.5
4.5
1
20
3
6
7.4
7.5
20.3
21.2
21.6
0.39
3.4
3.8
9
3.03
0.48
0.41
4.43
8
10.5
11.0
16.0
19.6
21.0
6.4
73
75
8.0
8.2
8.9
9.0
21.7
216
23.0
248
78
14.3
8.3
154
67.7
0.8
1.0
2.17
4.63
2.19
2.64
6.24
8.41
12
14
7
16
32
70
21
43
68
15
29
49
10
25
35
5
4.9
5.6
6.3
8.1
3.2
4.7
22
23
92
157
12.6
99
226
1.71
7.9
26
350
5.4
225
595
629
2.92
4.14
3.61
4.3
2.1
3.6
600
640
230
40.12
45.01
58.89
70.61
83.55
74.13
85.44
19.75
35.44
43.29
55.13
45.38
46.07
60
9.814
90.16
16.5
10.1
40
45
37.5
38.5
4.6
51.0
64
2.3
29.0
33.3
36.2
46.2
13.8
17.4
24.5
7.1
3.3
3.5
6.8
14.9
3.9
6.2
192
280
31
130
2
88
200
107
67.4
73.4
27.4
50.0
9.6
IC50 ≤ 6.5
IC50 ≥ 65
13.3
44.8
16.1
15.1
10.7
0.02
0.022
0

In [209]:
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(lambda x: x.split()[0] if x != '' else None)


def final_ic50_clean(s):

    try:
        return float(s)
    
    except:
        return None
        

final_ic50_clean('s')

In [210]:
dramp_nonclassic['ic50'] = dramp_nonclassic['ic50'].apply(final_ic50_clean)
dramp_nonclassic = dramp_nonclassic.dropna(subset=['ic50'])
dramp_nonclassic.shape

(1402, 13)

In [215]:
dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].fillna('Free')

In [218]:
# nterm
# Free                                        826+
# Acetylation                                 391+
# Not included yet                             76+
# Oct (Fatty acid 1-Octanoyl)                  54+
# lactam                                       32+
# Acetylization                                 7+
# Lactam                                        4+
# gu=N , N , N ', N'-tetramethylguanidino)      4+
# C9                                            3+
# 3-OH-4-Me-C15                                 2+

dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].apply(lambda x: x.strip())
dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].apply(lambda x: 'Acetylation' if x == 'Acetylization' else x)
dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].apply(lambda x: 'Free' if x == 'Not included yet' else x)
dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].apply(lambda x: 'lactam' if x == 'Lactam' else x)
dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].apply(lambda x: 'CH3(CH2)6CO' if x == 'Oct (Fatty acid 1-Octanoyl)' else x)
dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].apply(lambda x: 'tetramethylguanidine' if x == "gu=N , N , N ', N'-tetramethylguanidino)" else x)
dramp_nonclassic['nterm'] = dramp_nonclassic['nterm'].apply(lambda x: '3_OH_4_Me_C15' if x == '3-OH-4-Me-C15' else x)
dramp_nonclassic['nterm'].value_counts()

nterm
Free                    905
Acetylation             398
CH3(CH2)6CO              54
lactam                   36
tetramethylguanidine      4
C9                        3
3_OH_4_Me_C15             2
Name: count, dtype: int64

In [212]:
dramp_nonclassic['cterm'].value_counts()

cterm
Amidation                               698
Free                                    554
Not included yet                         76
lactam                                   32
Aminoethanol                             20
CH3(CH2)14CONH2, Hexadecanamide          12
Lactam                                    3
α-amidated residue at the C-terminus      3
Not available                             1
Name: count, dtype: int64

In [219]:
# cterm
# Amidation                               698+
# Free                                    554+
# Not included yet                         76+
# lactam                                   32+
# Aminoethanol                             20+
# CH3(CH2)14CONH2, Hexadecanamide          12+
# Lactam                                    3
# α-amidated residue at the C-terminus      3
# Not available                             1

dramp_nonclassic['cterm'] = dramp_nonclassic['cterm'].fillna('Free')
dramp_nonclassic['cterm'] = dramp_nonclassic['cterm'].apply(lambda x: x.strip())
dramp_nonclassic['cterm'] = dramp_nonclassic['cterm'].apply(lambda x: 'Free' if x =='Not included yet' else x)
dramp_nonclassic['cterm'] = dramp_nonclassic['cterm'].apply(lambda x: 'Free' if x =='Not available' else x)
dramp_nonclassic['cterm'] = dramp_nonclassic['cterm'].apply(lambda x: 'CH3(CH2)14CONH2' if x =='CH3(CH2)14CONH2, Hexadecanamide' else x)
dramp_nonclassic['cterm'] = dramp_nonclassic['cterm'].apply(lambda x: 'lactam' if x =='Lactam' else x)
dramp_nonclassic['cterm'] = dramp_nonclassic['cterm'].apply(lambda x: 'Amidation' if x =='α-amidated residue at the C-terminus' else x)
dramp_nonclassic['cterm'].value_counts()

cterm
Amidation          701
Free               634
lactam              35
Aminoethanol        20
CH3(CH2)14CONH2     12
Name: count, dtype: int64

In [213]:
dramp_nonclassic['topology'].value_counts()

topology
Linear              852
Cyclic              466
Not included yet     76
Tetrameric            6
Name: count, dtype: int64

In [220]:
dramp_nonclassic = dramp_nonclassic[dramp_nonclassic['topology'] != 'Tetrameric']
dramp_nonclassic['topology'] = dramp_nonclassic['topology'].fillna('Linear')
dramp_nonclassic['topology'] = dramp_nonclassic['topology'].apply(lambda x: 'Linear' if x == 'Not included yet' else x)
dramp_nonclassic['topology'].value_counts()

topology
Linear    930
Cyclic    466
Name: count, dtype: int64

In [221]:
dramp_nonclassic['mod'].unique()

array(['Not included yet',
       "Disulfide bond between Cys8 and Cys33, Cys12 and Cys29, Cys17 and Cys26; the 'Met' at position 10 is Methionine sulfoxide.",
       "Disulfide bond between Cys7 and Cys31, Cys11 and Cys27, Cys16 and Cys24; the 'Met' at position 5 and 9 is Methionine sulfoxide.",
       'Disulfide bond (Cys1-Cys16)', 'l=D-Leucine; k=D-Lysine',
       'X(2/13)=Leu(3-OH); X(5)=N(Me)xiIle; X(6)=Asn(3-OH); X(7/10)=xiThr; X(11)=Unk',
       'rrrrrrrr=D-isomer homopolymer of eight arginine residues',
       'X=Orn; f=D-Phe', 'X=Leu-S', 'X=Daa',
       'X(5)= 4-Hydroxyproline; X(8)=Daa', 'X=Dip (Diphenylalanine)',
       'X(1/2/5/6/9)=ornithine; X(4)=Dip (Diphenylalanine)',
       'X=Lys-Acetylation (Lysine with acetylation side group)',
       'X(11)=Pemetrexed', 'X(12)=Pemetrexed',
       'X(6)=AC6C=cyclohexyl-1-carboxylic acid', 'l=D-Leu', 'm=D-Met',
       'a=D-Ala', 'f=D-Phe', 'X=Ornithine', 'k=D-Lys',
       'X=Aib=1-amino-isobutyric acid', 'X=Nal=3-(1-naphthyl)alanine'

In [222]:
def get_ss_bridge(m):

    if m == "Disulfide bond between Cys8 and Cys33, Cys12 and Cys29, Cys17 and Cys26; the 'Met' at position 10 is Methionine sulfoxide.":
        return 'C8-C33;C12-C29;C17-C26'
    
    if m == "Disulfide bond between Cys7 and Cys31, Cys11 and Cys27, Cys16 and Cys24; the 'Met' at position 5 and 9 is Methionine sulfoxide.":
        return 'C7-C31;C11-C27;C16-C24'
    
    if m == "Disulfide bond (Cys1-Cys16)":
        return 'C1-C16'
    
    return None

dramp_nonclassic['ss_bridge'] = dramp_nonclassic['mod'].apply(get_ss_bridge)
dramp_nonclassic['ss_bridge'].value_counts()

ss_bridge
C1-C16                    5
C8-C33;C12-C29;C17-C26    1
C7-C31;C11-C27;C16-C24    1
Name: count, dtype: int64

In [ ]:
# соответствие
# Leu(3-OH)                                                 - nothing
# N(Me)xiIle                                                - xiIle
# Asn(3-OH)                                                 - nothing
# xiThr                                                     - xiThr
# Unk                                                       - nothing
# Orn/ornithine                                             - Orn
# Leu-S                                                     - nothing
# Daa                                                       - nothing
# 4-Hydroxyproline                                          - Hyp
# Dip (Diphenylalanine)                                     - Dip
# X=Lys-Acetylation (Lysine with acetylation side group)    - Lys_Ac
# Pemetrexed                                                - nothing
# 
#